In [1]:
import os
import tarfile
import urllib
import ssl
import pandas as pd
import numpy as np
from zlib import crc32
from sklearn.model_selection import train_test_split #원본 데이터셋에서 test set을 쉽게 나눌 수 있는 모듈
from sklearn.model_selection import StratifiedShuffleSplit #계층적 샘플링을 이용하여 test set을 만들 수 있는 모듈

from sklearn.impute import SimpleImputer #결측치를 쉽게 다룰 수 있는 모듈

from sklearn.preprocessing import OrdinalEncoder #카테고리(ordinal data)를 숫자형으로 바꿀 수 있는 모듈. 머신러닝 알고리즘은 주로 숫자를 다루기 때문에 사용함.
from sklearn.preprocessing import OneHotEncoder #원-핫 인코딩 할 수 있는 모듈

from sklearn.base import BaseEstimator, TransformerMixin #나만의 변환기를 만들기 위해 class에 상속하는 모듈들

from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler

from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestRegressor

"""
원하는 작업: 다른 측정 데이터가 주어졌을 때 구역의 중간 주택 가격을 예측하고자 함.
주어진 데이터: 캘리포니아 인구 조사 데이터 셋(중간 주택 가격, 인구, 중간 소득 등등의 데이터를 포함함)
만들어야 하는 모델: 캘리포니아 인구조사 데이터를 활용하여 캘리포니아의 주택 가격 모델 만들기
"""

"""
2.3 : 데이터 다운로드 및 데이터 살펴보기, test set 떼어내기
2.4 : 데이터 이해를 위한 탐색(상관계수)과 시각화
"""


DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path = HOUSING_PATH): #데이터를 다운로드 하는 함수
    os.makedirs(housing_path, exist_ok = True) #datasets/housing 디렉터리를 만듦
    tgz_path = os.path.join(housing_path, "housing.tgz") #tgz파일의 경로와 이름을 지정
    urllib.request.urlretrieve(housing_url, tgz_path) #파일 다운
    housing_tgz = tarfile.open(tgz_path) #파일 오픈
    housing_tgz.extractall(path=housing_path) #압축 해제
    housing_tgz.close() #종료
    
def load_housing_data(housing_path=HOUSING_PATH): #pandas를 활용하여 데이터프레임 읽어들이기
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

def split_train_test(data, test_ratio): #무작위로 샘플을 선택하어 데이터 셋의 특정 비율(보통 20%~30%)정도를 떼어놓는 함수.
    shuffled_indices = np.random.permutation(len(data)) #dataframe(array)를 복사한 후 섞음 cf)np.random.shuffle()의 경우 원본 array를 변화시킴. 여기서는 행의 번호 순서를 섞어놓는다고 생각하면 됨.
    test_set_size = int(len(data)*test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    # 이 방식은 프로그램을 실행할 때마다 test set이 변한다는 단점이 있음. -> 첫 시행에서의 test set을 저장하거나, 항상 같은 난수 index가 발생하도록 seed를 지정하는 방법 있음.
    # 근데 위 두가지 방식도 데이터 셋을 업데이트 한다면 문제가 발생함. -> 샘플의 식별자(각 샘플이 식별자 가졌다고 가정)를 사용하여 test set으로 보낼지 말지를 결정하도록 함.
    return data.iloc[train_indices], data.iloc[test_indices] #iloc은 행을 선택하는 방식임.

def test_set_check(identifier, test_ratio): #샘플의 식별자를 활용하여 기준에 충족하는지를 판단하는 함수.
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2 ** 32 #샘플의 해시값이 기준보다 작은 경우에만 test set으로 보냄.

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

def display_scores(scores):
    print("점수:", scores)
    print("평균:", scores.mean())
    print("표준편차:", scores.std())

In [2]:
fetch_housing_data() #SSLCertVerificationError 발생시 해결방법 : https://indesire.tistory.com/239
housing = load_housing_data()

In [3]:
housing["income_cat"] = pd.cut(housing["median_income"], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1,2,3,4,5]) #기존 특성을 분할하여 카테고리 1~5를 만듦.

# scikitlearn의 StratifiedShuffleSampling을 활용하면 계층적 샘플링을 쉽게 할 수 있음.
split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

strat_test_set["income_cat"].value_counts() / len(strat_test_set)

3    0.350533
2    0.318798
4    0.176357
5    0.114583
1    0.039729
Name: income_cat, dtype: float64

In [4]:
#income_cat 특성을 삭제하여 데이터를 원래 상태로 돌림.
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis =1 , inplace = True) #axis가 0일 경우 행을 삭제, 1일 경우 열을 삭제. inplace의 기본값은 False이나 True로 설정시 데이터프레임 자체를 수정후 아무런 값도 반환하지 않음.

In [5]:
housing = strat_train_set.drop("median_house_value", axis=1) #drop사용시 strat_train_set에는 영향을 미치지 않음. axis=1은 열을 삭제하는 옵션. default는 0.
housing_labels = strat_train_set["median_house_value"].copy() #예측 변수와 레이블을 분리하기 위해 median_house_value 열을 복사해줌.

# 2.5.1 데이터 정제

특성값이 누락될 경우 3가지 방법을 사용할 수 있음.(결측치 제거)
1. 해당 구역 제거
2. 전체 특성 삭제
3. 어떤 값(0, 평균값, 중간값)등으로 채움.

In [6]:
housing.dropna(subset=["total_bedrooms"]) # dropna(subset)사용시 특정 열을 기준으로 결측치가 있을 경우 해당 행을 삭제한다.
                                          # 참고: https://m.blog.naver.com/youji4ever/221712578078
    
housing.drop("total_bedrooms", axis = 1) #total_bedrooms 열 자체를 삭제함.

median = housing["total_bedrooms"].median() #total_bedrooms열에 해당하는 값들의 중간값을 median에 저장함.

housing["total_bedrooms"].fillna(median, inplace=True) #결측값을 median으로 채움. inplace=True 옵션은 원본 데이터셋을 보존하지 않고 바로 변경할 때 사용. 

In [7]:
imputer = SimpleImputer(strategy="median") #SimpleImputer 객체를 생성함.
housing_num = housing.drop("ocean_proximity", axis=1) #중간값은 수치형 특성에서만 계산 가능하므로 ocean_proximity 열을 삭제한 데이터 복사본 생성
imputer.fit(housing_num) #모든 특성의 중간값을 계산해서 그 결과값을 객체의 statistics_ 속성에 저장함. fit은 training set에 적용한다는 의미.
imputer.statistics_

array([-118.51  ,   34.26  ,   29.    , 2119.5   ,  433.    , 1164.    ,
        408.    ,    3.5409])

In [8]:
housing_num.median().values #위 imputer.statistics_와 동일한 값을 출력. numpy의 method를 활용한 것.

array([-118.51  ,   34.26  ,   29.    , 2119.5   ,  433.    , 1164.    ,
        408.    ,    3.5409])

In [9]:
X = imputer.transform(housing_num) #imputer 객체를 활용해서 누락된 값을 중간값으로 바꿈. 이때 X는 numpy array

In [10]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index) #X를 pandas DataFrame으로 간단히 바꿈.

# 2.5.2 텍스트와 범주형 특성 다루기

In [11]:
housing_cat = housing[["ocean_proximity"]] #범주형 특성이 ocean_proximity열만 따로 housing_cat에 저장함.
housing_cat.head(10)

,ocean_proximity
17606,<1H OCEAN
18632,<1H OCEAN
14650,NEAR OCEAN
3230,INLAND
3555,<1H OCEAN
19480,INLAND
8879,<1H OCEAN
13685,INLAND
4937,<1H OCEAN
4861,<1H OCEAN


In [12]:
ordinal_encoder = OrdinalEncoder() #OrdinalEncoder 객체 생성
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat) #OrdinalEncoder를 housing_cat에 적용시켜 categories_를 만들고 
                                                                 #categories_를 활용해 카테고리를 텍스트에서 숫자로 변환시킴.
housing_cat_encoded[:10]

array([[0.],
       [0.],
       [4.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.]])

In [13]:
ordinal_encoder.categories_ #카테고리 목록을 확인 할 수 있음. 범주형 특성마다 카테고리의 1차원 배열을 담은 리스트가 반환됨.

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

머신러닝 알고리즘의 경우 카테고리 0-1의 사이를 카테고리 0-4의 사이보다 가깝다고 생각함.
이는 카테고리 사이의 순서가 명확한 경우에는 문제가 되지 않음. ex) bad average good
그러나 카테고리 사이의 순서가 명확하지 않은 경우에는 알고리즘이 오해할 수 있음.
따라서 이럴때는 카테고리별 이진 특성을 만들어서 해당할 경우에 1, 해당하지 않을 경우에 0으로 표시하는 scipy sparse matrix(희소행렬)을 만들어서 활용함. -> OneHotEncoder 활용

In [14]:
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot 
#이 결과 출력이 희소행렬임. 특성이 수천 개일 경우 해당하지 않는 카테고리를 다 0으로 표시하면 메모리 비효율적임. 따라서 0이 아닌 원소의 위치만 저장하는 희소행렬이 매우 효율적.

<16512x5 sparse matrix of type '<class 'numpy.float64'>'
	with 16512 stored elements in Compressed Sparse Row format>

In [15]:
housing_cat_1hot.toarray() #희소행렬을 array로 바꾸고 싶을때 toarray()사용.

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

In [16]:
cat_encoder.categories_ #OneHotEncoder에서도 categories_를 사용하여 카테고리로 이뤄진 리스트를 얻을 수 있음.

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

# 2.5.3 나만의 변환기

OrdinalEncoder, OneHotEncoder 등이 있지만 내가 원하는 작업을 위해 나만의 변환기를 만들어야 할 때도 있음. <br>
이때 Scikitlearn의 기능과 연동되기 위해서는 fit(), transform(), fit_transform() 메서드를 포함하는 클래스를 만들어 주면된다.<br>
cf) fit_transfrom()은 TransformerMixin을 상속하면 자동으로 포함할 수 있음.<br>
cf) BaseEstimator를 상속한 후 생성자에 * args나 ** kargs를 사용하지 않으면 하이퍼 파라미터 튜닝에 필요한 get_params()와 set_params()메서드를 포함할 수 있음. 

In [17]:
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): #변환기가 add_bedrooms_per_room 하이퍼파라미터 하나를 가짐. 기본값은 True.
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

# 2.5.4 특성 스케일링

머신러닝 알고리즘의 경우 입력 숫자 특성들의 스케일이 많이 다르면 잘 작동하지 않음. -> min-max 스케일링(정규화) 또는 standardization(표준화)가 널리 사용됨.<br>
min-max 스케일리으이 경우 0~1의 범위에 들도록 값을 조정함. Scikitlearn의 MinMaxScaler 변환기를 활용할 수 있음. 0~1이 아닌 다른 범위를 원하면 feature_range 매개변수로 범위 변경 가능.<br>
표준화는 (값 - 평균값) / 표준편차 를 활용함. 범위에 상하한이 없어 문제가 될 수 있으나 이상치의 영향을 덜 받는 장점이 있음. Scikitlearn의 StandardScaler 변환기를 활용할 수 있음.<br>
cf) 스케일링은 전체 데이터가 아닌 training set에만 적용(fit())해야함. 그 후 training set, test set에 transform() 사용함.

# 2.5.5 변환 파이프라인

In [18]:
num_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")),
                        ('attribs_adder', CombinedAttributesAdder()),
                         ('std_scaler', StandardScaler()),
                        ])

In [19]:
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),
])
housing_prepared = full_pipeline.fit_transform(housing)

In [20]:
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared,housing_labels)

LinearRegression()

In [21]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("예측:", lin_reg.predict(some_data_prepared))
print("레이블:", list(some_labels))

예측: [210644.60459286 317768.80697211 210956.43331178  59218.98886849
 189747.55849879]
레이블: [286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


In [22]:
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

68628.19819848923

In [23]:
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

In [24]:
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [25]:
display_scores(tree_rmse_scores)

점수: [69016.71253399 66844.11444278 70811.13590686 69699.47285461
 70157.54756863 74852.66270992 71979.92046405 71476.35141803
 76803.14432363 70789.64700519]
평균: 71243.07092276754
표준편차: 2698.1779587773617


In [26]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

점수: [66782.73843989 66960.118071   70347.95244419 74739.57052552
 68031.13388938 71193.84183426 64969.63056405 68281.61137997
 71552.91566558 67665.10082067]
평균: 69052.46136345083
표준편차: 2731.674001798342


In [28]:
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

18684.539808589496

In [31]:
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)

In [32]:
display_scores(forest_rmse_scores)

점수: [49371.49707126 47476.90646097 49809.41141743 52530.90276471
 49613.32697901 53224.92085703 49266.78099746 48084.94955884
 52937.35222122 49975.93104674]
평균: 50229.19793746639
표준편차: 1898.9673258940766
